##### Imports:

In [13]:
from utils import calculate_covariance_from_chroma, separate_for_training, calculate_mu_from_chroma, calculate_transition_probabilites, format_indiv_chroma, predict, get_unique_predicted, calculate_initial_probabilities
import pickle
from chroma import get_chromagram
import pandas as pd
from tqdm import tqdm
import numpy as np
from hmmlearn import hmm
from sklearn.metrics import f1_score

##### Steps:

1. Training / Testing Data Split
2. Create Chromagram from Training Data
3. Create HMM Initialization Components
    - Initial State Probabilities
    - Transition Probability Matrix
    - Mu Value
    - Emission Matrix
4. Create HMM Object
5. Fit / Train HMM

##### Training / Test Data Split:

In [6]:
# Load data and split into training and test
with open(r"dataset.pkl", 'rb') as data:
    midi_data:dict = pickle.load(data)

training_piece_names, test_piece_names = separate_for_training(midi_data, 0.8)
NOTES_NAMES =   ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]
FULL_CHORD_LIST = [note + suffix for note in NOTES_NAMES for suffix in ['', 'm', 'dim']]

##### Create Chromagram from Training Data:

In [7]:
song_chromagrams = []
for song_name in tqdm(list(training_piece_names)):
    indiv_chroma = get_chromagram(song_name, midi_data)
    formatted = format_indiv_chroma(indiv_chroma)
    song_chromagrams.append(indiv_chroma)

chromagram = pd.concat(song_chromagrams)
chromagram.head(200)

100%|███████████████████████████████████████| 4609/4609 [00:50<00:00, 91.89it/s]


,C,C#,D,D#,E,F,F#,G,G#,A,A#,B,Chord Actual
0,0,0,111,0,0,0,50,0,0,0,0,49,Bm
1,0,0,49,0,0,0,50,0,0,0,0,49,Bm
2,0,0,49,0,0,0,50,0,0,0,0,49,Bm
3,0,0,49,0,0,0,50,0,0,0,0,49,Bm
4,0,0,0,0,0,0,58,0,0,0,0,0,Bm
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,0,0,0,0,47,0,0,54,0,0,0,103,Em
2,0,0,0,0,47,0,0,54,0,0,0,50,Em
3,0,0,0,0,47,0,0,54,0,0,0,50,Em
4,0,0,0,0,47,0,0,54,0,0,0,50,Em


##### Create HMM Components:

###### Initial State Probabilities:

In [55]:
initial_state_probabilties = calculate_initial_probabilities(training_piece_names, midi_data)
initial_state_probabilties

C        0.000193
Cm       0.000000
Cdim     0.000000
C#       0.000964
C#m      0.058438
C#dim    0.000000
D        0.121311
Dm       0.000000
Ddim     0.000000
D#       0.000193
D#m      0.020444
D#dim    0.000000
E        0.116297
Em       0.060752
Edim     0.000000
F        0.000000
Fm       0.000000
Fdim     0.000000
F#       0.020058
F#m      0.021215
F#dim    0.000000
G        0.118997
Gm       0.000000
Gdim     0.000000
G#       0.005593
G#m      0.156027
G#dim    0.000000
A        0.020636
Am       0.000000
Adim     0.000000
A#       0.000579
A#m      0.000000
A#dim    0.000000
B        0.121697
Bm       0.156606
Bdim     0.000000
dtype: float64

###### Transition Matrix:

In [56]:
transition_prob_matrix = calculate_transition_probabilites(chromagram)
print(transition_prob_matrix)
assert np.allclose(transition_prob_matrix.sum(axis=1), 1), "Not all rows sum to 1"

              C        Cm      Cdim        C#       C#m     C#dim         D  \
C      0.861736  0.000000  0.000000  0.001608  0.000000  0.000000  0.003215   
Cm     0.027778  0.027778  0.027778  0.027778  0.027778  0.027778  0.027778   
Cdim   0.027778  0.027778  0.027778  0.027778  0.027778  0.027778  0.027778   
C#     0.000000  0.000000  0.000000  0.863343  0.000430  0.000000  0.006016   
C#m    0.000000  0.000000  0.000000  0.000000  0.858921  0.000000  0.003067   
C#dim  0.027778  0.027778  0.027778  0.027778  0.027778  0.027778  0.027778   
D      0.000314  0.000000  0.000000  0.000000  0.001725  0.000000  0.863529   
Dm     0.027778  0.027778  0.027778  0.027778  0.027778  0.027778  0.027778   
Ddim   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
D#     0.000000  0.000000  0.000000  0.001749  0.001749  0.000000  0.006997   
D#m    0.000000  0.000000  0.000000  0.008675  0.003429  0.000000  0.006254   
D#dim  0.027778  0.027778  0.027778  0.027778  0.027

###### Mu Value:

In [57]:
mu = calculate_mu_from_chroma(chromagram)

###### Covariance Matrix:

In [58]:
# Example usage
covars = calculate_covariance_from_chroma(chromagram)
print("Covariances shape:", covars.shape)

Covariances shape: (36, 12, 12)


In [59]:
model = hmm.GaussianHMM(n_components=transition_prob_matrix.shape[0], covariance_type="diag")
model.startprob_ = initial_state_probabilties
model.transmat_ = transition_prob_matrix.values
model.means_ = mu.reshape(-1, 1)
model.covars_ = np.array([np.diag(cov_matrix) + 1e-6 for cov_matrix in covars]).reshape(-1, 12)
model.n_features = 36

In [73]:
custom_encoding = {}
for i, chord in enumerate(FULL_CHORD_LIST):
    custom_encoding[chord] = i

true_labels = []
predicted_labels = []
for song_name in tqdm(list(test_piece_names)):
    last_chord, chromagram_without_last_chord = separate_last_chord(get_chromagram(song_name, midi_data))
    if not chromagram_without_last_chord.empty:
        encoded_chromagram_without_last_chord = chromagram_without_last_chord['Chord Actual'].apply(lambda x: custom_encoding.get(x, -1)).values.reshape(-1, 1)
        preds = model.predict(encoded_chromagram_without_last_chord)
        prediction = preds[-1]
        predicted_labels.append(prediction)
        true_labels.append(custom_encoding.get(last_chord, -1))

f1 = f1_score(true_labels, predicted_labels, average='micro')
print(f"F1 Score: {f1}")

100%|████████████████████████████████████████| 577/577 [00:05<00:00, 104.00it/s]

F1 Score: 0.0658578856152513
